We first get necessary external data and code

In [1]:
!git clone https://github.com/AllenInstitute/deepinterpolation.git
!mkdir -p ephys

fatal: destination path 'deepinterpolation' already exists and is not an empty directory.
�T�u�f�B���N�g���܂��̓t�@�C�� -p �͊��ɑ��݂��܂��B
�������ɃG���[���������܂���: -p
�T�u�f�B���N�g���܂��̓t�@�C�� ephys �͊��ɑ��݂��܂��B
�������ɃG���[���������܂���: ephys


Install deepinterpolation package

In [2]:
!pip install git+https://github.com/AllenInstitute/deepinterpolation.git

  Cloning https://github.com/AllenInstitute/deepinterpolation.git to c:\users\hyperpc_smee\appdata\local\temp\pip-req-build-lkgsf4ka
  Resolved https://github.com/AllenInstitute/deepinterpolation.git to commit 8057088a0d6b4b1889014ffaa2f107fc2d88896a
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/AllenInstitute/deepinterpolation.git 'C:\Users\HyperPC_Smee\AppData\Local\Temp\pip-req-build-lkgsf4ka'


In [32]:
import os
from deepinterpolation.generator_collection import SingleTifGenerator
from deepinterpolation.trainor_collection import core_trainer
from deepinterpolation.network_collection import unet_single_1024
import datetime

import tkinter.filedialog

This is used for record-keeping


In [33]:
now = datetime.datetime.now()
run_uid = now.strftime("%Y_%m_%d_%H_%M")

Initialize meta-parameters objects

In [34]:
training_param = {}
generator_param = {}
network_param = {}
generator_test_param = {}

An epoch is defined as the number of batches pulled from the dataset. Because our datasets are VERY large. Often, we cannot
go through the entirity of the data so we define an epoch slightly differently than is usual.

In [35]:
steps_per_epoch = 10
pre_post_frame = 7

Those are parameters used for the Validation test generator. Here the test is done on the beginning of the data but
this can be a separate file.


In [36]:
# Those are parameters used for the Validation test generator. Here the
# test is done on the beginning of the data but
# this can be a separate file
generator_test_param[
    "pre_post_frame"
] = pre_post_frame  # Number of frame provided before and after the predicted frame
filePath = tkinter.filedialog.askopenfilename()
generator_test_param["train_path"] = filePath
generator_test_param["batch_size"] = 5
generator_test_param["start_frame"] = 0
generator_test_param["end_frame"] = 99
generator_test_param[
    "pre_post_omission"
] = 1  # Number of frame omitted before and after the predicted frame
generator_test_param[
    "steps_per_epoch"
] = -1
# No step necessary for testing as epochs are not relevant.
# -1 deactivate it.

In [38]:
generator_param["steps_per_epoch"] = steps_per_epoch
generator_param[
    "pre_post_frame"
] = generator_test_param["pre_post_frame"]  # Number of frame provided before and after the predicted frame
# In practice the test and validation would point at different files or different portion of files. But for demo, we have little data available

generator_param["train_path"] = filePath
generator_param["batch_size"] = 1
generator_param["start_frame"] = 0
generator_param["end_frame"] = 99
generator_param[
    "pre_post_omission"
] = 0  # Number of frame omitted before and after the predicted frame
generator_param["steps_per_epoch"] = -1  # No step necessary for testing as epochs are not relevant. -1 deactivate it.

Those are parameters used for the main data generator

In [39]:
generator_param["train_path"]

'D:/deepinterpolation/sample_data/C2-reged_651-08.0w-D1-xyt6-300.tif'

Those are parameters used for the network topology

Those are parameters used for the training process

In [40]:
training_param["run_uid"] = run_uid
training_param["batch_size"] = generator_test_param["batch_size"]
training_param["steps_per_epoch"] = steps_per_epoch
training_param[
    "period_save"
] = 25  # network model is potentially saved during training between a regular nb epochs
training_param["nb_gpus"] = 0
training_param["apply_learning_decay"] = 0
training_param[
    "nb_times_through_data"
] = 1  # if you want to cycle through the entire data. Two many iterations will cause noise overfitting
training_param["learning_rate"] = 0.0001
training_param["pre_post_frame"] = generator_test_param["pre_post_frame"]
training_param["loss"] = "mean_absolute_error"
training_param[
    "nb_workers"
] = 1  # this is to enable multiple threads for data generator loading. Useful when this is slower than training

training_param["model_string"] = (
    "unet_single_1024_"
    + training_param["loss"]
    + "_"
    + training_param["run_uid"]
)

Where do you store ongoing training progress

In [41]:
jobdir = os.path.join(
    training_param["model_string"] + "_" + run_uid,
)
training_param["output_dir"] = jobdir

try:
    os.mkdir(jobdir)
except:
    print("folder already exists")

In [42]:
training_param["output_dir"]

'unet_single_1024_mean_absolute_error_2024_10_22_16_49_2024_10_22_16_49'

Here we create all json files that are fed to the training. This is used for recording purposes as well as input to the training proces

Here we create all objects for training.

In [43]:
# We build the generators object. This will, among other things, calculate normalizing parameters.
train_generator = SingleTifGenerator(generator_param)
test_generator = SingleTifGenerator(generator_test_param)

# We build the training object.
training_class = core_trainer(train_generator, test_generator, unet_single_1024({}), training_param)

Start training. This can take very long time.

In [44]:
training_class.run()

Epoch 1/9
10/10 [==============================] - 13s 665ms/step - loss: 1.3805 - val_loss: 0.3340
Epoch 2/9
10/10 [==============================] - 6s 65ms/step - loss: 0.3455 - val_loss: 0.3278
Epoch 3/9
10/10 [==============================] - 5s 51ms/step - loss: 0.3373 - val_loss: 0.3221
Epoch 4/9
10/10 [==============================] - 5s 52ms/step - loss: 0.3288 - val_loss: 0.3132
Epoch 5/9
10/10 [==============================] - 5s 51ms/step - loss: 0.3181 - val_loss: 0.3039
Epoch 6/9
10/10 [==============================] - 6s 66ms/step - loss: 0.3104 - val_loss: 0.2999
Epoch 7/9
10/10 [==============================] - 6s 66ms/step - loss: 0.3071 - val_loss: 0.2957
Epoch 8/9
10/10 [==============================] - 6s 69ms/step - loss: 0.3042 - val_loss: 0.2939
Epoch 9/9
10/10 [==============================] - 6s 64ms/step - loss: 0.3020 - val_loss: 0.2920


Finalize and save output of the training.

In [45]:
training_class.finalize()

Saved model to disk


In [16]:
training_param

{'run_uid': '2024_10_11_17_44',
 'batch_size': 1,
 'steps_per_epoch': 5,
 'period_save': 25,
 'nb_gpus': 0,
 'apply_learning_decay': 0,
 'nb_times_through_data': 1,
 'learning_rate': 0.0001,
 'pre_post_frame': 30,
 'loss': 'mean_absolute_error',
 'nb_workers': 1,
 'model_string': 'unet_single_1024_mean_absolute_error_2024_10_11_17_44',
 'output_dir': 'unet_single_1024_mean_absolute_error_2024_10_11_17_44_2024_10_11_17_44'}